In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import unicodedata
import re
import random

# Loading data

In [14]:
SOS_token = 0
EOS_token = 1

class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {SOS_token: 'SOS', EOS_token: 'EOS'}
        self.n_words = 2    # for SOS and EOS tokens
        
    def add_sentence(self, sentence):
        for word in sentence.split(' '):
            self.add_word(word)
            
    def add_word(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [15]:
def unicode_to_ascii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters
def normalize_string(s):
    s = unicode_to_ascii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z!?]+", r" ", s)
    return s.strip()

In [16]:
def read_langs(lang1: str, lang2: str, reverse=False):
    lines = open("data/%s-%s.txt" % (lang1, lang2), encoding="utf-8").readlines()
    pairs = [[normalize_string(s) for s in l.split('\t')] for l in lines]
    
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)
        
    return input_lang, output_lang, pairs

In [17]:
input_lang, output_lang, pairs = read_langs('eng', 'fra')
for pair in pairs:
    input_lang.add_sentence(pair[0])
    output_lang.add_sentence(pair[1])

In [18]:
MAX_LENGTH = 100

# Processing data

In [19]:
def indexes_from_sentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]

def tensor_from_sentence(lang, sentence):
    indexes = indexes_from_sentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long).view(1, -1)

def tensors_from_pair(pair):
    input_tensor = tensor_from_sentence(input_lang, pair[0])
    target_tensor = tensor_from_sentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

In [22]:
input_lang, output_lang, pairs = read_langs('eng', 'fra', True)
for pair in pairs:
    input_lang.add_sentence(pair[0])
    output_lang.add_sentence(pair[1])
    
print(random.choice(pairs))

n = len(pairs)
input_ids = torch.zeros((n, MAX_LENGTH), dtype=torch.int32)
target_ids = torch.zeros((n, MAX_LENGTH), dtype=torch.int32)

for idx, (inp, tgt) in enumerate(pairs):
    inp_ids = indexes_from_sentence(input_lang, inp)
    tgt_ids = indexes_from_sentence(output_lang, tgt)
    inp_ids.append(EOS_token)
    tgt_ids.append(EOS_token)
    input_ids[idx, :len(inp_ids)] = torch.Tensor(inp_ids)
    target_ids[idx, :len(tgt_ids)] = torch.Tensor(tgt_ids)

['les randonneurs doivent emporter une boussole avec eux pour trouver leur chemin a travers les bois', 'hikers need to carry a compass with them to find their way through the woods']
